# An integrated oemof constraint: Emission limit
As emissions are often to be limited, a constraint limiting the energy systems emissions has been introduced as an oemof feature.

This example is taken from git: /oemof/oemof-examples-master/oemof_examples/oemof.solph/v0.3.x/emission_constraint/emission_constraint.py

All necessary packages are imported:

In [1]:
import pandas as pd

import oemof.solph as solph
from oemof.solph import constraints

Basic definition of energy system model and buses of the energy system:

In [2]:
energysystem = solph.EnergySystem(
                    timeindex=pd.date_range('1/1/2012', periods=3, freq='H'))

bgas = solph.Bus(label="gas")

bel = solph.Bus(label="electricity", balanced=False)

energysystem.add(bel, bgas)

Now, all components to which emissions are connected are added to the energy system with a specific "emission factor":

In [3]:
energysystem.add(solph.Sink(label='demand',
                              inputs={
                                bel: solph.Flow(actual_value=[10, 20, 30],
                                                fixed=True)}))

energysystem.add(solph.Source(label='biomass',
                              outputs={
                                bel: solph.Flow(nominal_value=100,
                                                variable_costs=10,
                                                emission_factor=0.01,
                                                actual_value=[0.1, 0.2, 0.3],
                                                fixed=True)}))

energysystem.add(solph.Source(label='gas-source',
                              outputs={
                                bel: solph.Flow(variable_costs=10,
                                                emission_factor=0.2)}))

energysystem.add(solph.Transformer(
    label="pp_gas",
    inputs={
        bgas: solph.Flow()},
    outputs={
        bel: solph.Flow(nominal_value=200)},
    conversion_factors={
        bel: 0.58}))

When all components are added, the energy system model is generated with oemof, using pyomo. This creates all linear equations describing the energy system model. When solving this model, the solver (often cbc) reads these equations and finds the solution with the lowest objective value.

In [4]:
model = solph.Model(energysystem)

Now, the linear equations are stored into a file. 

In [5]:
model.write('./1_lp_file_oemof.lp', io_options={'symbolic_solver_labels': True})

lp_file_text = open('./1_lp_file_oemof.lp')
for line in lp_file_text:
    print(line)

\* Source Pyomo model name=Model *\



min 

objective:

+10 flow(gas_source_electricity_0)

+10 flow(gas_source_electricity_1)

+10 flow(gas_source_electricity_2)

+600 ONE_VAR_CONSTANT



s.t.



c_e_Bus_balance(gas_0)_:

+1 flow(gas_pp_gas_0)

= 0



c_e_Bus_balance(gas_1)_:

+1 flow(gas_pp_gas_1)

= 0



c_e_Bus_balance(gas_2)_:

+1 flow(gas_pp_gas_2)

= 0



c_e_Transformer_relation(pp_gas_gas_electricity_0)_:

+1 flow(gas_pp_gas_0)

-1.7241379310344829 flow(pp_gas_electricity_0)

= 0



c_e_Transformer_relation(pp_gas_gas_electricity_1)_:

+1 flow(gas_pp_gas_1)

-1.7241379310344829 flow(pp_gas_electricity_1)

= 0



c_e_Transformer_relation(pp_gas_gas_electricity_2)_:

+1 flow(gas_pp_gas_2)

-1.7241379310344829 flow(pp_gas_electricity_2)

= 0



c_e_ONE_VAR_CONSTANT: 

ONE_VAR_CONSTANT = 1.0



bounds

   0 <= flow(gas_pp_gas_0) <= +inf

   0 <= flow(gas_pp_gas_1) <= +inf

   0 <= flow(gas_pp_gas_2) <= +inf

   0 <= flow(gas_source_electricity_0) <= +inf

   0 <= flow(gas_source_

Now, the emission constraint is added from the oemof library:

In [6]:
constraints.emission_limit(model, limit=100)

model.write('./1_lp_file_oemof_emission_constraint.lp', io_options={'symbolic_solver_labels': True})

lp_file_text = open('./1_lp_file_oemof_emission_constraint.lp')
for line in lp_file_text:
    print(line)

\* Source Pyomo model name=Model *\



min 

objective:

+10 flow(gas_source_electricity_0)

+10 flow(gas_source_electricity_1)

+10 flow(gas_source_electricity_2)

+600 ONE_VAR_CONSTANT



s.t.



c_u_emission_limit_:

+0.20000000000000001 flow(gas_source_electricity_0)

+0.20000000000000001 flow(gas_source_electricity_1)

+0.20000000000000001 flow(gas_source_electricity_2)

<= 99.400000000000006



c_e_Bus_balance(gas_0)_:

+1 flow(gas_pp_gas_0)

= 0



c_e_Bus_balance(gas_1)_:

+1 flow(gas_pp_gas_1)

= 0



c_e_Bus_balance(gas_2)_:

+1 flow(gas_pp_gas_2)

= 0



c_e_Transformer_relation(pp_gas_gas_electricity_0)_:

+1 flow(gas_pp_gas_0)

-1.7241379310344829 flow(pp_gas_electricity_0)

= 0



c_e_Transformer_relation(pp_gas_gas_electricity_1)_:

+1 flow(gas_pp_gas_1)

-1.7241379310344829 flow(pp_gas_electricity_1)

= 0



c_e_Transformer_relation(pp_gas_gas_electricity_2)_:

+1 flow(gas_pp_gas_2)

-1.7241379310344829 flow(pp_gas_electricity_2)

= 0



c_e_ONE_VAR_CONSTANT: 

ONE_VAR_

In [7]:
model.total_emissions.pprint()

model.solve()

total_emissions : Size=1, Index=None
    Key  : Expression
    None : 0.01*flow[biomass,electricity,0] + 0.01*flow[biomass,electricity,1] + 0.01*flow[biomass,electricity,2] + 0.2*flow[gas-source,electricity,0] + 0.2*flow[gas-source,electricity,1] + 0.2*flow[gas-source,electricity,2]


{'Problem': [{'Name': 'unknown', 'Lower bound': 600.0, 'Upper bound': 600.0, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 10, 'Number of nonzeros': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.029686927795410156}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Testing whether constraint was fullfilled:

In [8]:
print(model.total_emissions())

0.6000000000000001
